In [ ]:
import scipy.io
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats

# Full Python code including data loading, processing, percentile calculation, and plotting with correct mapping

# Load the MAT file data
file_path = '../Data/05_Granger_TE.mat'
data = scipy.io.loadmat(file_path)

# Function to create DataFrame with task data, significance, and subset labels
def create_dataframe(task_predictive_influence, task_nonpredictive_influence, sig_predictive, sig_nonpredictive, label):
    task_combined = np.concatenate((task_predictive_influence, task_nonpredictive_influence))
    sig_combined = np.concatenate((sig_predictive, sig_nonpredictive))
    # labels = [f'Predictive → Nonpredictive {label}'] * len(task_predictive_influence) + [f'Nonpredictive → Predictive {label}'] * len(task_nonpredictive_influence)
    labels = [f'P → N {label}'] * len(task_predictive_influence) + [f'N → P {label}'] * len(task_nonpredictive_influence)

    df = pd.DataFrame({
        'Value': task_combined,
        'Sig': sig_combined,
        'Label': labels
    })
    df['Color'] = df['Sig'].apply(lambda x: 'red' if x == 1 else 'blue')
    return df

df_granger_spa = create_dataframe(data['spaGrangerSubsetPredNotSubset'][0], data['spaGrangerNotSubsetPredSubset'][0],
                                  data['spaGrangerSubsetPredNotSubsetSig'][0], data['spaGrangerNotSubsetPredSubsetSig'][0], '\n(Spatial)')
df_granger_cue = create_dataframe(data['cueGrangerSubsetPredNotSubset'][0], data['cueGrangerNotSubsetPredSubset'][0],
                                  data['cueGrangerSubsetPredNotSubsetSig'][0], data['cueGrangerNotSubsetPredSubsetSig'][0], '\n(Cue)')
df_TE_spa = create_dataframe(data['spaTransferEntropySubsetPredNotSubset'][0], data['spaTransferEntropyNotSubsetPredSubset'][0],
                                  data['spaTransferEntropySubsetPredNotSubsetSig'][0], data['spaTransferEntropyNotSubsetPredSubsetSig'][0], '\n(Spatial)')
df_TE_cue = create_dataframe(data['cueTransferEntropySubsetPredNotSubset'][0], data['cueTransferEntropyNotSubsetPredSubset'][0],
                                  data['cueTransferEntropySubsetPredNotSubsetSig'][0], data['cueTransferEntropyNotSubsetPredSubsetSig'][0], '\n(Cue)')

df_granger = pd.concat([df_granger_spa, df_granger_cue], ignore_index=True)
df_TE = pd.concat([df_TE_spa, df_TE_cue], ignore_index=True)

color_mapping = {'red': 'red', 'blue': 'blue'}

# Revised approach to ensure correct plotting of 95th percentile lines

# Function to calculate the 95th percentile for each 'Null' variable
def calculate_percentiles_null(data):
    percentiles = []
    ordered_keys = ['spaGrangerSubsetPredNotSubsetNull', 'spaGrangerNotSubsetPredSubsetNull',
                    'cueGrangerSubsetPredNotSubsetNull', 'cueGrangerNotSubsetPredSubsetNull',
                    'spaTransferEntropySubsetPredNotSubsetNull', 'spaTransferEntropyNotSubsetPredSubsetNull',
                    'cueTransferEntropySubsetPredNotSubsetNull', 'cueTransferEntropyNotSubsetPredSubsetNull']
    for key in ordered_keys:
        percentiles.append(np.percentile(data[key][0], 95))
    return percentiles

# Calculate the 95th percentiles for the 'Null' variables
percentiles_null = calculate_percentiles_null(data)

# Function to plot with 95th percentile lines
def plot_with_percentiles(df, percentiles, ax, title):
    sns.swarmplot(x='Label', y='Value', hue='Color', data=df, ax=ax, palette=color_mapping, size=14, dodge=False)
    ax.set_xticks(range(len(df['Label'].unique())))
    # ax.set_xticklabels(df['Label'].unique(), rotation=45, ha='right')
    ax.set_xticklabels([label.replace('P', r'$\mathregular{\mathit{P}}$').replace('N', r'$\mathregular{\mathit{N}}$') for label in df['Label'].unique()])
    ax.set_xlabel('')
    ax.set_ylabel(title, fontsize=36)
    ax.set_ylim(top=0.7)
    ax.tick_params(axis='both', which='major', labelsize=18)
    # ax.get_legend().remove()
    
    # Adding custom legend
    legend_elements = [
        plt.Line2D([0], [0], marker='o', color='w', label='p < 0.05', markerfacecolor='red', markersize=10),
        plt.Line2D([0], [0], marker='o', color='w', label='p ≥ 0.05', markerfacecolor='blue', markersize=10),
        plt.Line2D([0], [0], color='black', lw=1, linestyle='--', label='95th percentile'),
        plt.Line2D([0], [0], marker='$P$', color='black', markerfacecolor='black', markersize=10, lw=0, label='Predictive\nSubset'),
        plt.Line2D([0], [0], marker='$N$', color='black', markerfacecolor='black', markersize=10, lw=0, label='Nonpredictive\nSubset')
    ]
    
    # ax.legend(handles=legend_elements, fontsize='12', loc='upper right')
    ax.legend(
        handles=legend_elements,
        fontsize='12',
        loc='upper right'
    )
    
    # labels_helper = ['Predictive → Nonpredictive (Spatial Task)', 'Nonpredictive → Predictive (Spatial Task)', 'Predictive → Nonpredictive (Cue Task)', 'Nonpredictive → Predictive (Cue Task)']
    labels_helper = ['P → N\n(Spatial)', 'N → P\n(Spatial)', 'P → N\n(Cue)', 'N → P\n(Cue)']
    
    label_positions = np.arange(len(labels_helper))
    
    for i, label in enumerate(labels_helper):
        # Plot the 95th percentile of the null distribution as a dashed line within the label area
        if title == "Transfer Entropy":
            ax.hlines(y=percentiles[i + 4], xmin=label_positions[i] - 0.2, xmax=label_positions[i] + 0.2,
                      colors='black', linestyles='--')
        else:
            ax.hlines(y=percentiles[i], xmin=label_positions[i] - 0.2, xmax=label_positions[i] + 0.2,
                      colors='black', linestyles='--')

# Create a figure with two subplots
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Plotting Granger values with 95th percentile lines
plot_with_percentiles(df_granger, percentiles_null, axes[0], 'Granger value')

# Plotting Transfer Entropy values with 95th percentile lines
plot_with_percentiles(df_TE, percentiles_null, axes[1], 'Transfer Entropy')

# Adjust layout and show the plot
plt.tight_layout()
plt.show()